In [270]:
import pandas as pd
import ast
import re
import json
#from difflib import SequenceMatcher
#from geopy.distance import geodesic
#import numpy as np
import duckdb
import time

In [271]:
# Import sample data
sample_df = pd.read_parquet('project_b_samples_2k.parquet')
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""shipping_center"",""alternate"":[""fre...",0.996261,"[""http://www.goinpostaljacksonville.com/""]","[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[{""dataset"":""Microsoft"",""record_id"":""168884986...","{""primary"":""Goin' Postal Jacksonville""}","{""primary"":""vehicle_shipping"",""alternate"":[""co...",0.77,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{""freeform"":""7643 Gate Pkwy Ste 104"",""localit..."
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Valley Transmission""}","{""primary"":""automotive_repair"",""alternate"":[""a...",0.996261,"[""http://valleytransca.com/""]","[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[{""dataset"":""Microsoft"",""record_id"":""112589990...","{""primary"":""Valley Transmission""}","{""primary"":""transmission_repair"",""alternate"":[...",0.77,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{""freeform"":""1158 N 2nd St"",""locality"":""El Ca..."
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Mazda Nelspruit""}","{""primary"":""car_dealer"",""alternate"":[""automoti...",0.997899,"[""http://bit.ly/NelspruitMazda""]","[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424934...","{""primary"":""Mazda Nelspruit""}","{""primary"":""used_car_dealer"",""alternate"":[""aut...",0.77,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{""freeform"":""49 Emnotweni Avenue"",""locality"":..."
3,1407374885933937,844424932146366,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Red Wing - Roswell, GA""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.770000,"[""https://stores.redwingshoes.com/roswell-ga""]",None,None,"[""7705584400""]",...,"[{""dataset"":""Microsoft"",""record_id"":""844424932...","{""primary"":""Red Wing""}","{""primary"":""shoe_store"",""alternate"":[""online_s...",0.77,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{""freeform"":""875 Mansell Rd Ste B1"",""locality..."
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Norauto España""}","{""primary"":""automotive_repair"",""alternate"":[""c...",0.996261,"[""https://centro.norauto.es/alacant/finestrat/...","[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[{""dataset"":""Microsoft"",""record_id"":""140737488...","{""primary"":""Norauto""}","{""primary"":""brake_service_and_repair"",""alterna...",0.77,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{""freeform"":""Calle Rei Jaume I 9"",""locality"":..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{""property"":"""",""dataset"":""meta"",""record_id"":""...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""lawyer"",""alternate"":[""criminal_def...",0.286713,"[""http://www.petraccimarin.it""]","[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[{""property"":"""",""dataset"":""FourSquare"",""record...","{""primary"":""Studio Legale Associato Petracci M...","{""primary"":""Business 

In [272]:
# For previewing the data, look at first data point in sample_df
first_data_point = sample_df.iloc[0]

print("Sources value:")
print(first_data_point['names'])
print(type(first_data_point['names']))

print("\nParsed sources:")
sources_data = json.loads(first_data_point['names'])
for i, source in enumerate(sources_data):
    print(f"Source {i+1}: {source}")

Sources value:
{"primary":"Goin' Postal Jacksonville"}
<class 'str'>

Parsed sources:
Source 1: primary


In [273]:
# Load the Yelp business dataset
with open('Yelp JSON/yelp_dataset/yelp_academic_dataset_business.json', 'r') as file:
    businesses = []
    for line in file:
        business = json.loads(line.strip())
        businesses.append(business)

print(f"Number of records: {len(businesses)}")
print("Sample business record:")
print(json.dumps(businesses[0], indent=2))

Number of records: 150346
Sample business record:
{
  "business_id": "Pns2l4eNsfO8kk83dixA6A",
  "name": "Abby Rappoport, LAC, CMQ",
  "address": "1616 Chapala St, Ste 2",
  "city": "Santa Barbara",
  "state": "CA",
  "postal_code": "93101",
  "latitude": 34.4266787,
  "longitude": -119.7111968,
  "stars": 5.0,
  "review_count": 7,
  "is_open": 0,
  "attributes": {
    "ByAppointmentOnly": "True"
  },
  "categories": "Doctors, Traditional Chinese Medicine, Naturopathic/Holistic, Acupuncture, Health & Medical, Nutritionists",
  "hours": null
}


In [274]:
yelp_df = pd.DataFrame(businesses)

print(f"DataFrame shape: {yelp_df.shape}")
print("\nDataFrame columns:")
print(yelp_df.columns.tolist())
yelp_df.head()

DataFrame shape: (150346, 14)

DataFrame columns:
['business_id', 'name', 'address', 'city', 'state', 'postal_code', 'latitude', 'longitude', 'stars', 'review_count', 'is_open', 'attributes', 'categories', 'hours']


,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."


In [275]:
# Check current types
sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'str'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1996,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1997,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'NoneType'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>
1998,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>


In [276]:
# Assigns data types to attributes
def parse_category(x):
    if isinstance(x, (list, dict)) or pd.isna(x): return x
    try: return ast.literal_eval(x)
    except: return None

for c in ["sources", "names", "addresses","base_names","base_addresses","base_categories", "categories", "websites", "brand"]:
    if c in sample_df: sample_df[c] = sample_df[c].apply(parse_category)

sample_df.map(type)

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_sources,base_names,base_categories,base_confidence,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses
0,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
1,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
2,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
3,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
4,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'NoneType'>,<class 'NoneType'>,<class 'str'>,<class 'str'>,<class 'list'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1996,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1997,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'NoneType'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>
1998,<class 'str'>,<class 'str'>,<class 'list'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'list'>,<class 'str'>,<class 'NoneType'>,<class 'str'>,...,<class 'str'>,<class 'dict'>,<class 'dict'>,<class 'float'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'list'>


## Normalize and Extract Important Information
Let's create some helper functions to normalize names, and extract crucial address information

In [277]:
def norm_name(s):
    s = re.sub(r"\s+", " ", (s or "").strip().lower())
    s = re.sub(r"\b(llc|inc|corp|co|ltd|the)\b", "", s)
    s = re.sub(r"[^a-z0-9\s&\-']", " ", s)
    return re.sub(r"\s+", " ", s).strip()

def extract_postcode(addr_list):
    if isinstance(addr_list, list) and addr_list:
        pc = str(addr_list[0].get("postcode") or None)
        if pc and pc != 'None':
            # Remove ZIP+4 extension (e.g., "49007-1293" becomes "49007")
            pc = pc.split('-')[0].strip()
        return pc

def extract_city(addr_list):
    if isinstance(addr_list, list) and addr_list:
        city_str = addr_list[0].get("locality") or None
        if city_str:
            return city_str.strip().lower()
        return city_str

def extract_name(name_list):
    if isinstance(name_list, dict) and name_list:
        return name_list.get("primary")

In [278]:
sample_df["name_candidate"] = sample_df["names"].apply(extract_name)
sample_df["name_norm"] = sample_df["name_candidate"].fillna("").apply(norm_name)
sample_df["zip"] = sample_df["addresses"].apply(extract_postcode)
sample_df["city_norm"] = sample_df["addresses"].apply(extract_city)

In [279]:
sample_df

,id,base_id,sources,names,categories,confidence,websites,socials,emails,phones,...,base_websites,base_socials,base_emails,base_phones,base_brand,base_addresses,name_candidate,name_norm,zip,city_norm
0,08f44f055a9a016e0390f050aa3c93c0,1688849865669487,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Goin' Postal Jacksonville'},"{'primary': 'shipping_center', 'alternate': ['...",0.996261,[http://www.goinpostaljacksonville.com/],"[""https://www.facebook.com/463273470392736""]",None,"[""+19049989600""]",...,"[""https://www.goinpostaljacksonville.com/""]",None,None,"[""9049989600""]","{""names"":{}}","[{'freeform': '7643 Gate Pkwy Ste 104', 'local...",Goin' Postal Jacksonville,goin' postal jacksonville,32256,jacksonville
1,08f29a456e42e5830324637954145c50,1125899907111860,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Valley Transmission'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[http://valleytransca.com/],"[""https://www.facebook.com/107150256087765""]",None,"[""+16194474353""]",...,"[""http://valleytransca.com/""]",None,None,"[""6194474353""]","{""names"":{}}","[{'freeform': '1158 N 2nd St', 'locality': 'El...",Valley Transmission,valley transmission,92021,el cajon
2,08fbcd0030da5323031bcafa8c2fa0dc,844424934845986,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Mazda Nelspruit'},"{'primary': 'car_dealer', 'alternate': ['autom...",0.997899,[http://bit.ly/NelspruitMazda],"[""https://www.facebook.com/113070635372645""]",None,"[""+27137573800""]",...,"[""https://nelspruitmazda.co.za/""]",None,None,"[""0137573800""]","{""names"":{}}","[{'freeform': '49 Emnotweni Avenue', 'locality...",Mazda Nelspruit,mazda nelspruit,1201,riverside park
3,1407374885933937,844424932146366,"[{'dataset': 'Microsoft', 'record_id': '140737...","{'primary': 'Red Wing - Roswell, GA'}","{'primary': 'shoe_store', 'alternate': ['onlin...",0.770000,[https://stores.redwingshoes.com/roswell-ga],None,None,"[""7705584400""]",...,"[""http://stores.redwing.com/roswell-ga""]",None,None,"[""7705584400""]","{""names"":{}}","[{'freeform': '875 Mansell Rd Ste B1', 'locali...","Red Wing - Roswell, GA",red wing - roswell ga,30076,roswell
4,08f3956260b9e14003feca2bf0764d0c,1407374887472291,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Norauto España'},"{'primary': 'automotive_repair', 'alternate': ...",0.996261,[https://centro.norauto.es/alacant/finestrat/c...,"[""https://www.facebook.com/295967644448975""]",None,"[""+34966803254""]",...,"[""http://centro.norauto.es/centre/alicante-fin...",None,None,"[""966803254""]","{""names"":{}}","[{'freeform': 'Calle Rei Jaume I 9', 'locality...",Norauto España,norauto espa a,03509,finestrat
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1995,08f1e1251b0a66c003a195f0bf6b4bc9,502a184a011ce8eabd5934fb,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Studio Legale Associato Petracci ...,"{'primary': 'lawyer', 'alternate': ['criminal_...",0.286713,[http://www.petraccimarin.it],"[""https://www.facebook.com/121776774503515""]",None,"[""+39040660022""]",...,"[""http://www.petraccimarin.it""]","[null,null,null]",[null],"[""040 660022""]","{""wikidata"":"""",""names"":{""primary"":""Studio Lega...","[{'freeform': 'Via beccaria 3', 'locality': 'T...",Studio Legale Associato Petracci Marin,studio legale associato petracci marin,34125,trieste
1996,08fa80b035674b6a03379ca9c1ecea9f,50364256e4b0573431043dd4,"[{'property': '', 'dataset': 'meta', 'record_i...",{'primary': 'Farmácias Pague Menos'},"{'primary': 'pharmacy', 'alternate': ['pharmac...",0.792793,[https://www.paguemenos.com.br/],"[""https://www.facebook.com/2204406756483378""]",None,"[""+556740028282""]",...,"[""https://www.paguemenos.com.br""]","[null,null,null]",[null],"[""(67) 3521-4016""]","{""wikidata"":"""",""names"":{""primary"":""Farmacias P...","[{'freeform': 'Av. Antônio Trajano, 583', 'loc...",Farmácias Pague Menos,farm cias pague menos,7

In [280]:
# Extract country information from addresses
def extract_country(addr_list):
    """Extract country from address list"""
    if isinstance(addr_list, list) and addr_list:
        country = addr_list[0].get("country")
        if country:
            return country.strip().upper()
    return None

# Add country column to sample_df
sample_df["country"] = sample_df["addresses"].apply(extract_country)

# Count US records
us_records = sample_df[sample_df["country"] == "US"]
print(f"Number of US records: {len(us_records)}")
print(f"Total records: {len(sample_df)}")
print(f"Percentage of US records: {len(us_records)/len(sample_df)*100:.1f}%")

# Show country distribution
print(f"\nCountry distribution:")
country_counts = sample_df["country"].value_counts()
print(country_counts)

Number of US records: 960
Total records: 2000
Percentage of US records: 48.0%

Country distribution:
country
US    960
DE    151
IT    110
BR     95
GB     94
JP     90
MX     75
TH     73
FR     70
CA     51
IN     44
AU     27
ES     22
AT     18
NL     10
MY     10
TR      9
ID      8
DK      7
CH      7
PL      7
SE      6
CZ      4
BE      4
PT      4
PH      3
AR      3
NZ      3
HU      3
FI      3
ZA      3
NO      2
PR      2
RU      2
IE      2
CO      2
OM      2
SG      1
CL      1
DO      1
BO      1
SK      1
GR      1
PE      1
LV      1
IQ      1
KR      1
CN      1
VN      1
RS      1
SR      1
Name: count, dtype: int64


In [281]:
yelp_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"Doctors, Traditional Chinese Medicine, Naturop...",None
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"Shipping Centers, Local Services, Notaries, Ma...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ..."
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","Department Stores, Shopping, Fashion, Home & G...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ..."
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","Restaurants, Food, Bubble Tea, Coffee & Tea, B...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ..."
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","Brewpubs, Breweries, Food","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","Nail Salons, Beauty & Spas","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3..."
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","Pets, Nurseries & Gardening, Pet Stores, Hobby...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3..."
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","Shopping, Jewelry, Piercing, Toy Stores, Beaut...",None
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","Fitness/Exercise Equipment, Eyewear & Optician...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ..."


In [282]:
yelp_df.map(type)

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours
0,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'NoneType'>
1,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>
2,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>
3,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>
4,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>
150342,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>
150343,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'NoneType'>
150344,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'str'>,<class 'float'>,<class 'float'>,<class 'float'>,<class 'int'>,<class 'int'>,<class 'dict'>,<class 'str'>,<class 'dict'>


In [283]:
# Parse JSON string columns that might be strings instead of dicts/lists
def parse_yelp_attributes(x):
    """Parse attributes column which might be a string representation of dict"""
    if isinstance(x, dict) or pd.isna(x): 
        return x
    if isinstance(x, str):
        try: 
            return ast.literal_eval(x)
        except: 
            return {}
    return {}

def parse_yelp_hours(x):
    """Parse hours column which might be a string representation of dict"""
    if isinstance(x, dict) or pd.isna(x): 
        return x
    if isinstance(x, str):
        try: 
            return ast.literal_eval(x)
        except: 
            return {}
    return {}

def parse_yelp_categories(x):
    """Parse categories - split string into list"""
    if pd.isna(x) or x is None:
        return []
    if isinstance(x, list):
        return x
    if isinstance(x, str):
        # Yelp categories are comma-separated strings
        return [cat.strip() for cat in x.split(',') if cat.strip()]
    return []

print("Parsing Yelp data types...")
# Apply parsing to relevant columns
if 'attributes' in yelp_df.columns:
    yelp_df['attributes'] = yelp_df['attributes'].apply(parse_yelp_attributes)
if 'hours' in yelp_df.columns:
    yelp_df['hours'] = yelp_df['hours'].apply(parse_yelp_hours)
if 'categories' in yelp_df.columns:
    yelp_df['categories'] = yelp_df['categories'].apply(parse_yelp_categories)

print("Data type parsing complete!")

Parsing Yelp data types...
Data type parsing complete!


In [284]:
# Create normalized address structure for Yelp data
def create_yelp_address_dict(row):
    """Create standardized address dict from Yelp business data"""
    return {
        'address': row.get('address', ''),
        'city': row.get('city', ''),
        'state': row.get('state', ''),
        'postcode': row.get('postal_code', ''),
        'country': 'US'  # Yelp data is primarily US-based
    }

def extract_yelp_zip(address_dict):
    """Extract zip code from Yelp address"""
    if isinstance(address_dict, dict):
        pc = str(address_dict.get('postcode', '') or '')
        # Extract 5-digit zip code
        m = re.search(r'(\d{5})', pc)
        return m.group(1) if m else None
    return None

def extract_yelp_city(address_dict):
    """Extract normalized city from Yelp address"""
    if isinstance(address_dict, dict):
        city = address_dict.get('city', '') or ''
        return city.strip().lower() if city else None
    return None

def extract_yelp_state(address_dict):
    """Extract state from Yelp address"""
    if isinstance(address_dict, dict):
        state = address_dict.get('state', '') or ''
        return state.strip().upper() if state else None
    return None

# Apply address normalization
print("Creating normalized address structure...")
yelp_df['address_dict'] = yelp_df.apply(create_yelp_address_dict, axis=1)

# Extract key fields using your normalization functions
print("Extracting and normalizing key fields...")
yelp_df['name_norm'] = yelp_df['name'].fillna('').apply(norm_name)
yelp_df['zip'] = yelp_df['address_dict'].apply(extract_yelp_zip)
yelp_df['city_norm'] = yelp_df['address_dict'].apply(extract_yelp_city)
yelp_df['state'] = yelp_df['address_dict'].apply(extract_yelp_state)

print("Normalization complete!")

Creating normalized address structure...
Extracting and normalizing key fields...
Normalization complete!


In [285]:
yelp_df

,business_id,name,address,city,state,postal_code,latitude,longitude,stars,review_count,is_open,attributes,categories,hours,address_dict,name_norm,zip,city_norm
0,Pns2l4eNsfO8kk83dixA6A,"Abby Rappoport, LAC, CMQ","1616 Chapala St, Ste 2",Santa Barbara,CA,93101,34.426679,-119.711197,5.0,7,0,{'ByAppointmentOnly': 'True'},"[Doctors, Traditional Chinese Medicine, Naturo...",None,"{'address': '1616 Chapala St, Ste 2', 'city': ...",abby rappoport lac cmq,93101,santa barbara
1,mpf3x-BjTdTEA3yCZrAYPw,The UPS Store,87 Grasso Plaza Shopping Center,Affton,MO,63123,38.551126,-90.335695,3.0,15,1,{'BusinessAcceptsCreditCards': 'True'},"[Shipping Centers, Local Services, Notaries, M...","{'Monday': '0:0-0:0', 'Tuesday': '8:0-18:30', ...","{'address': '87 Grasso Plaza Shopping Center',...",ups store,63123,affton
2,tUFrWirKiKi_TAnsVWINQQ,Target,5255 E Broadway Blvd,Tucson,AZ,85711,32.223236,-110.880452,3.5,22,0,"{'BikeParking': 'True', 'BusinessAcceptsCredit...","[Department Stores, Shopping, Fashion, Home & ...","{'Monday': '8:0-22:0', 'Tuesday': '8:0-22:0', ...","{'address': '5255 E Broadway Blvd', 'city': 'T...",target,85711,tucson
3,MTSW4McQd7CbVtyjqoe9mw,St Honore Pastries,935 Race St,Philadelphia,PA,19107,39.955505,-75.155564,4.0,80,1,"{'RestaurantsDelivery': 'False', 'OutdoorSeati...","[Restaurants, Food, Bubble Tea, Coffee & Tea, ...","{'Monday': '7:0-20:0', 'Tuesday': '7:0-20:0', ...","{'address': '935 Race St', 'city': 'Philadelph...",st honore pastries,19107,philadelphia
4,mWMc6_wTdE0EUBKIGXDVfA,Perkiomen Valley Brewery,101 Walnut St,Green Lane,PA,18054,40.338183,-75.471659,4.5,13,1,"{'BusinessAcceptsCreditCards': 'True', 'Wheelc...","[Brewpubs, Breweries, Food]","{'Wednesday': '14:0-22:0', 'Thursday': '16:0-2...","{'address': '101 Walnut St', 'city': 'Green La...",perkiomen valley brewery,18054,green lane
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150341,IUQopTMmYQG-qRtBk-8QnA,Binh's Nails,3388 Gateway Blvd,Edmonton,AB,T6J 5H2,53.468419,-113.492054,3.0,13,1,"{'ByAppointmentOnly': 'False', 'RestaurantsPri...","[Nail Salons, Beauty & Spas]","{'Monday': '10:0-19:30', 'Tuesday': '10:0-19:3...","{'address': '3388 Gateway Blvd', 'city': 'Edmo...",binh's nails,None,edmonton
150342,c8GjPIOTGVmIemT7j5_SyQ,Wild Birds Unlimited,2813 Bransford Ave,Nashville,TN,37204,36.115118,-86.766925,4.0,5,1,"{'BusinessAcceptsCreditCards': 'True', 'Restau...","[Pets, Nurseries & Gardening, Pet Stores, Hobb...","{'Monday': '9:30-17:30', 'Tuesday': '9:30-17:3...","{'address': '2813 Bransford Ave', 'city': 'Nas...",wild birds unlimited,37204,nashville
150343,_QAMST-NrQobXduilWEqSw,Claire's Boutique,"6020 E 82nd St, Ste 46",Indianapolis,IN,46250,39.908707,-86.065088,3.5,8,1,"{'RestaurantsPriceRange2': '1', 'BusinessAccep...","[Shopping, Jewelry, Piercing, Toy Stores, Beau...",None,"{'address': '6020 E 82nd St, Ste 46', 'city': ...",claire's boutique,46250,indianapolis
150344,mtGm22y5c2UHNXDFAjaPNw,Cyclery & Fitness Center,2472 Troy Rd,Edwardsville,IL,62025,38.782351,-89.950558,4.0,24,1,"{'BusinessParking': '{'garage': False, 'street...","[Fitness/Exercise Equipment, Eyewear & Opticia...","{'Monday': '9:0-20:0', 'Tuesday': '9:0-20:0', ...","{'address': '2472 Troy Rd', 'city': 'Edwardsvi...",cyclery & fitness center,62025,edwardsville


In [286]:
# Extract country from Yelp data (Yelp is primarily US-based)
def extract_yelp_country(address_dict):
    """Extract country from Yelp address dict"""
    if isinstance(address_dict, dict):
        country = address_dict.get('country', 'US')  # Default to US for Yelp data
        return country.strip().upper() if country else 'US'
    return 'US'

yelp_df["country"] = yelp_df["address_dict"].apply(extract_yelp_country)

# Count US records in Yelp data
us_records_yelp = yelp_df[yelp_df["country"] == "US"]
print(f"Number of US records in Yelp: {len(us_records_yelp)}")
print(f"Total Yelp records: {len(yelp_df)}")
print(f"Percentage of US records in Yelp: {len(us_records_yelp)/len(yelp_df)*100:.1f}%")

# Show country distribution in Yelp data
print(f"\nYelp country distribution:")
country_counts_yelp = yelp_df["country"].value_counts()
print(country_counts_yelp)

Number of US records in Yelp: 150346
Total Yelp records: 150346
Percentage of US records in Yelp: 100.0%

Yelp country distribution:
country
US    150346
Name: count, dtype: int64


## DuckDB Query

In [287]:
# Create a DuckDB connection
conn = duckdb.connect()

print("DuckDB installed and connection established!")
print(f"DuckDB version: {duckdb.__version__}")

DuckDB installed and connection established!
DuckDB version: 1.4.1


In [288]:
# Register  dataframes with DuckDB

conn.register('sample_df', sample_df)
conn.register('yelp_df', yelp_df)

print(f"Sample records: {len(sample_df)}")
print(f"Yelp records: {len(yelp_df)}")

result = conn.execute("SELECT COUNT(*) FROM sample_df").fetchone()
print(f"DuckDB can see {result[0]} records in sample_df")

Sample records: 2000
Yelp records: 150346
DuckDB can see 2000 records in sample_df


In [289]:
# Optimized DuckDB query w/ pre-filtering
optimized_query = """
WITH sample_prepared AS (
    SELECT 
        ROW_NUMBER() OVER () as sample_idx,
        name_norm as sample_name,
        city_norm as sample_city,
        zip as sample_zip
    FROM sample_df
    WHERE name_norm IS NOT NULL 
      AND name_norm != ''
      AND name_norm != 'None'
      AND LENGTH(name_norm) >= 3  -- Skip very short names
),
yelp_prepared AS (
    SELECT 
        ROW_NUMBER() OVER () as yelp_idx,
        name_norm as yelp_name,
        city_norm as yelp_city,
        zip as yelp_zip
    FROM yelp_df
    WHERE name_norm IS NOT NULL 
      AND name_norm != ''
      AND name_norm != 'None'
      AND LENGTH(name_norm) >= 3  -- Skip very short names
),
-- MUCH BETTER: Use strategic joins instead of CROSS JOIN
city_matches AS (
    SELECT s.*, y.*,
           1.0 as city_similarity,  -- Perfect city match
           CASE 
               WHEN LENGTH(s.sample_name) = 0 OR LENGTH(y.yelp_name) = 0 THEN 0.0
               ELSE 1.0 - (LEVENSHTEIN(s.sample_name, y.yelp_name) * 1.0 / 
                          GREATEST(LENGTH(s.sample_name), LENGTH(y.yelp_name)))
           END as name_similarity,
           CASE WHEN s.sample_zip = y.yelp_zip AND s.sample_zip IS NOT NULL THEN 1.0 ELSE 0.0 END as zip_match
    FROM sample_prepared s
    JOIN yelp_prepared y ON s.sample_city = y.yelp_city  -- EXACT city match
    WHERE LEVENSHTEIN(s.sample_name, y.yelp_name) <= 15  -- Reasonable edit distance
),
zip_matches AS (
    SELECT s.*, y.*,
           CASE 
               WHEN s.sample_city IS NULL AND y.yelp_city IS NULL THEN 1.0
               WHEN s.sample_city = y.yelp_city THEN 1.0
               WHEN s.sample_city IS NULL OR y.yelp_city IS NULL THEN 0.5
               WHEN LENGTH(s.sample_city) = 0 OR LENGTH(y.yelp_city) = 0 THEN 0.3
               ELSE 1.0 - (LEVENSHTEIN(s.sample_city, y.yelp_city) * 1.0 / 
                          GREATEST(LENGTH(s.sample_city), LENGTH(y.yelp_city)))
           END as city_similarity,
           CASE 
               WHEN LENGTH(s.sample_name) = 0 OR LENGTH(y.yelp_name) = 0 THEN 0.0
               ELSE 1.0 - (LEVENSHTEIN(s.sample_name, y.yelp_name) * 1.0 / 
                          GREATEST(LENGTH(s.sample_name), LENGTH(y.yelp_name)))
           END as name_similarity,
           1.0 as zip_match  -- Perfect ZIP match
    FROM sample_prepared s
    JOIN yelp_prepared y ON s.sample_zip = y.yelp_zip  -- EXACT zip match
    WHERE s.sample_zip IS NOT NULL 
      AND y.yelp_zip IS NOT NULL
      AND LEVENSHTEIN(s.sample_name, y.yelp_name) <= 8
),
-- Combine the results
all_matches AS (
    SELECT * FROM city_matches
    UNION ALL
    SELECT * FROM zip_matches
),
ranked_matches AS (
    SELECT *,
        -- Weighted similarity score
        (0.6 * name_similarity + 0.3 * city_similarity + 0.1 * zip_match) as overall_score,
        
        -- Rank matches for each sample record
        ROW_NUMBER() OVER (
            PARTITION BY sample_idx 
            ORDER BY (0.6 * name_similarity + 0.3 * city_similarity + 0.1 * zip_match) DESC
        ) as match_rank
    FROM all_matches
    WHERE (0.6 * name_similarity + 0.3 * city_similarity + 0.1 * zip_match) >= 0.4
)
SELECT * 
FROM ranked_matches 
WHERE match_rank <= 3  -- Top 3 matches per sample record
ORDER BY sample_idx, match_rank;
"""

In [290]:
# Testing query

print("Testing optimized DuckDB query...")
start_time = time.time()

# Execute the optimized query
optimized_matches = conn.execute(optimized_query).df()

optimized_time = time.time() - start_time

print(f"OPTIMIZED DUCKDB RESULTS:")
print(f"  • Time taken: {optimized_time:.2f} seconds")
print(f"  • Matches found: {len(optimized_matches)}")
if optimized_time > 0:
    print(f"  • Records per second: {len(optimized_matches)/optimized_time:.0f}")

if len(optimized_matches) > 0:
    print(f"\nSample matches:")
    print(optimized_matches[['sample_name', 'yelp_name', 'overall_score', 'name_similarity', 
                           'city_similarity', 'sample_city', 'yelp_city']].head())
    
    print(f"\nScore distribution:")
    print(f"Mean score: {optimized_matches['overall_score'].mean():.3f}")
    print(f"Min score: {optimized_matches['overall_score'].min():.3f}")
    print(f"Max score: {optimized_matches['overall_score'].max():.3f}")
else:
    print("No matches found with optimized query.")

Testing optimized DuckDB query...
OPTIMIZED DUCKDB RESULTS:
  • Time taken: 0.19 seconds
  • Matches found: 334
  • Records per second: 1758

Sample matches:
         sample_name      yelp_name  overall_score  name_similarity  \
0             garage         target       0.600000         0.500000   
1             garage           wawa       0.500000         0.333333   
2             garage           wawa       0.500000         0.333333   
3  commerce bank atm  commerce bank       0.858824         0.764706   
4  commerce bank atm  commerce bank       0.858824         0.764706   

   city_similarity    sample_city      yelp_city  
0              1.0     burlington     burlington  
1              1.0     burlington     burlington  
2              1.0     burlington     burlington  
3              1.0  saint charles  saint charles  
4              1.0  saint charles  saint charles  

Score distribution:
Mean score: 0.614
Min score: 0.406
Max score: 1.000


In [291]:
# Alternative: Even simpler and faster DuckDB approach
# Focus on exact and near-exact matches only

simple_fast_query = """
WITH sample_clean AS (
    SELECT 
        ROW_NUMBER() OVER () as sample_idx,
        name_norm as sample_name,
        city_norm as sample_city,
        zip as sample_zip
    FROM sample_df
    WHERE name_norm IS NOT NULL 
      AND name_norm != ''
),
yelp_clean AS (
    SELECT 
        ROW_NUMBER() OVER () as yelp_idx,
        name_norm as yelp_name,
        city_norm as yelp_city,
        zip as yelp_zip
    FROM yelp_df
    WHERE name_norm IS NOT NULL 
      AND name_norm != ''
)
-- Only look for very close matches to avoid expensive computations
SELECT 
    s.sample_idx,
    y.yelp_idx,
    s.sample_name,
    y.yelp_name,
    s.sample_city,
    y.yelp_city,
    s.sample_zip,
    y.yelp_zip,
    -- Simple similarity calculation
    1.0 - (LEVENSHTEIN(s.sample_name, y.yelp_name) * 1.0 / 
           GREATEST(LENGTH(s.sample_name), LENGTH(y.yelp_name))) as name_similarity,
    CASE WHEN s.sample_city = y.yelp_city THEN 1.0 ELSE 0.0 END as city_match,
    CASE WHEN s.sample_zip = y.yelp_zip THEN 1.0 ELSE 0.0 END as zip_match
FROM sample_clean s
JOIN yelp_clean y ON (
    -- Join condition: either exact city match or exact zip match
    s.sample_city = y.yelp_city OR s.sample_zip = y.yelp_zip
)
WHERE LEVENSHTEIN(s.sample_name, y.yelp_name) <= 5  -- Very close names only
ORDER BY s.sample_idx, name_similarity DESC
"""

In [292]:
# Test the simple fast query
print("Testing simple fast DuckDB query...")
start_time = time.time()

simple_matches = conn.execute(simple_fast_query).df()

simple_time = time.time() - start_time

print(f"SIMPLE FAST DUCKDB RESULTS:")
print(f"  • Time taken: {simple_time:.3f} seconds")
print(f"  • Matches found: {len(simple_matches)}")

if len(simple_matches) > 0:
    print(f"\nSample matches:")
    print(simple_matches[['sample_name', 'yelp_name', 'name_similarity', 
                         'sample_city', 'yelp_city', 'city_match']].head())

Testing simple fast DuckDB query...
SIMPLE FAST DUCKDB RESULTS:
  • Time taken: 1.167 seconds
  • Matches found: 548

Sample matches:
  sample_name yelp_name  name_similarity sample_city   yelp_city  city_match
0      garage    target         0.500000  burlington  burlington         1.0
1      garage      wawa         0.333333  burlington  burlington         1.0
2      garage      wawa         0.333333  burlington  burlington         1.0
3      garage      wawa         0.333333  burlington  burlington         1.0
4      garage      wawa         0.333333  burlington  burlington         1.0
